In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from src.causal_inference.causal_engine import FBRUSCausalAnalyzer,logger
# 设置绘图风格
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [2]:
# 主程序
# def main():
"""主函数"""
# 定义门店位置（美国主要城市的邮编）
store_locations = [
    {'name': 'FBR NYC Manhattan', 'postal_code': '10001'},     # 纽约曼哈顿
    {'name': 'FBR LA Downtown', 'postal_code': '90012'},       # 洛杉矶市中心
    {'name': 'FBR Chicago Loop', 'postal_code': '60601'},      # 芝加哥
    {'name': 'FBR SF Financial', 'postal_code': '94105'},      # 旧金山金融区
    {'name': 'FBR Miami Beach', 'postal_code': '33139'}        # 迈阿密海滩
]

# 创建分析器
analyzer = FBRUSCausalAnalyzer(store_locations)

try:
    # 1. 加载或生成销售数据
    sales_data = analyzer.load_sales_data()  # 使用模拟数据
    logger.info(f"销售数据形状: {sales_data.shape}")

    # 2. 获取天气数据
    weather_data = analyzer.fetch_weather_data()
    logger.info(f"天气数据形状: {weather_data.shape if weather_data is not None else 'None'}")

    # 3. 获取节假日数据
    holiday_data = analyzer.fetch_holiday_data()
    logger.info(f"节假日数据形状: {holiday_data.shape if holiday_data is not None else 'None'}")

    # 4. 合并所有数据
    merged_data = analyzer.merge_all_data()
    logger.info(f"合并数据形状: {merged_data.shape}")

    # 5. 执行因果分析
    causal_results = analyzer.perform_causal_analysis()

    # 6. 执行反事实分析
    counterfactual_results = analyzer.perform_counterfactual_analysis()

    # 7. 生成可视化报告
    analyzer.visualize_results(causal_results, counterfactual_results)

    # 8. 打印一些关键结果
    print("\n" + "="*60)
    print("FBR美国门店因果分析 - 关键发现")
    print("="*60)

    # 打印促销效果
    if 'promotion_effect' in causal_results and 'average_effect' in causal_results['promotion_effect']:
        print(f"\n📊 促销效果：{causal_results['promotion_effect']['interpretation']}")

    # 打印最优条件
    if 'optimal_conditions' in counterfactual_results:
        print(f"\n🎯 最优销售条件将带来：${counterfactual_results['optimal_conditions']['expected_sales']:.0f} 的销售额")

    # 打印主要风险
    if 'risk_scenarios' in counterfactual_results and counterfactual_results['risk_scenarios']:
        top_risk = counterfactual_results['risk_scenarios'][0]
        print(f"\n⚠️  最大风险：{top_risk['scenario']} - 预计损失 ${top_risk['predicted_loss']:.0f}")

    print("\n✅ 分析完成！")
    print("📄 报告文件：")
    print("   - 可视化报告: fbr_us_causal_analysis_report.html")
    print("   - 文字报告: fbr_us_causal_analysis_report.txt")

except Exception as e:
    logger.error(f"分析过程中出错: {e}", exc_info=True)
    raise

# return analyzer, causal_results, counterfactual_results
#
#
# if __name__ == "__main__":
#     # 运行分析
#     analyzer, causal_results, counterfactual_results = main()

2025-07-31 10:59:16,074 - INFO - 加载销售数据...
2025-07-31 10:59:16,076 - INFO - 使用模拟数据...
2025-07-31 10:59:16,086 - INFO - 生成了 1830 条销售记录
2025-07-31 10:59:16,087 - INFO - 销售数据形状: (1830, 10)
2025-07-31 10:59:16,087 - INFO - 获取天气数据...
2025-07-31 10:59:16,090 - INFO - Generating weather features for 5 postal codes.
2025-07-31 10:59:25,390 - INFO - Finished creating weather features. Total rows: 1830
2025-07-31 10:59:25,401 - INFO - 获取了 1830 条天气记录
2025-07-31 10:59:25,401 - INFO - 天气数据形状: (1830, 18)
2025-07-31 10:59:25,402 - INFO - 获取节假日数据...
2025-07-31 10:59:25,405 - INFO - Generating holiday features for 5 postal codes.
2025-07-31 10:59:25,537 - INFO - Finished creating holiday features. Total rows: 1830
2025-07-31 10:59:25,540 - INFO - 获取了 1830 条节假日记录
2025-07-31 10:59:25,541 - INFO - 节假日数据形状: (1830, 9)
2025-07-31 10:59:25,541 - INFO - 合并数据...
2025-07-31 10:59:25,549 - INFO - 合并后数据集包含 1830 条记录，32 个特征
2025-07-31 10:59:25,549 - INFO - 合并数据形状: (1830, 32)
2025-07-31 10:59:25,550 - INFO - 开始因果分析..


分析完成！

# FBR美国门店销售数据因果分析报告

生成时间：2025-07-31 10:59:40

## 一、执行摘要

本报告基于FBR美国门店的销售数据，结合天气和节假日信息，进行了深入的因果分析。

### 门店覆盖：
- FBR NYC Manhattan (邮编: 10001)
- FBR LA Downtown (邮编: 90012)
- FBR Chicago Loop (邮编: 60601)
- FBR SF Financial (邮编: 94105)
- FBR Miami Beach (邮编: 33139)

### 关键发现：


2. **促销效果**：
   - 促销平均提升销售额 $10504，ROI约为 -14.6%
   - 异质性效应：
     * 工作日: $7389
     * 周末: $8609
     * 晴天: $7517

3. **节假日效应**：
   - 节假日整体提升销售额 $nan
   - 购物季效应：购物季销售额平均减少 $nan

## 二、反事实分析

### 场景分析：

**完美促销日：节假日+好天气+促销**
- 预测销售额：$57491
- 相比平均值：$+2997

**最差情况：极端天气+无促销**
- 预测销售额：$50695
- 相比平均值：$-3798

**雨天促销策略**
- 预测销售额：$57491
- 相比平均值：$+2997

**黑色星期五场景**
- 预测销售额：$58168
- 相比平均值：$+3675

**普通工作日**
- 预测销售额：$49648
- 相比平均值：$-4845

### 最优运营条件：
- 预期销售额：$69857
- 建议：
  * 实施促销活动
  * 充分利用节假日流量
  * 最佳销售日是周六
  * 理想温度约70°F

### 风险场景预警：

- **连续雨天+工作日**
  * 预计损失：$4845
  * 风险等级：中
  * 缓解措施：雨天专属优惠、加强线上推广、改善店内体验

- **极端高温(>100°F)+无促销**
  * 预计损失：$3798
  * 风险等级：中
  * 缓解措施：加强空调、提供冷饮优惠、延长营业时间至晚上

- **暴风雪天气**
  * 预计损失：$3798
  * 风险等级：中

In [5]:
sales_data

,date,store_name,postal_code,sales_revenue,orders_count,has_promotion,discount_rate,inventory_level,day_of_week,is_weekend
0,2024-07-31 10:59:16.076790,FBR NYC Manhattan,10001,52483.570765,602.106902,0,0.000000,0.542270,2,False
1,2024-08-01 10:59:16.076790,FBR NYC Manhattan,10001,57006.632090,711.497004,1,0.228716,0.982927,3,False
2,2024-08-02 10:59:16.076790,FBR NYC Manhattan,10001,53582.658807,665.164544,0,0.000000,0.748680,4,False
3,2024-08-03 10:59:16.076790,FBR NYC Manhattan,10001,77327.500690,980.517886,1,0.246318,0.875736,5,True
4,2024-08-04 10:59:16.076790,FBR NYC Manhattan,10001,59654.862248,782.664900,0,0.000000,0.392767,6,True
...,...,...,...,...,...,...,...,...,...,...
1825,2025-07-27 10:59:16.076790,FBR Miami Beach,33139,46710.318381,591.819913,0,0.000000,0.984946,6,True
1826,2025-07-28 10:59:16.076790,FBR Miami Beach,33139,54410.679070,698.766808,1,0.196143,0.711903,0,False
1827,2025-07-29 10:59:16.076790,FBR Miami Beach,33139,55641.624020,703.427774,0,0.000000,0.689866,1,False
1828,2025-07-30 10:59:16.076790,FBR Miami Beach,33139,45789.909738,566.120328,0,0.000000,0.759057,2,False


In [7]:
holiday_data.groupby("is_holiday").count()

,date,postal_code,holiday_name,days_since_last_holiday,days_until_next_holiday,state,is_major_holiday,is_shopping_season
is_holiday,,,,,,,,
False,1765,1765,1765,1765,1765,1765,1765,1765
True,65,65,65,65,65,65,65,65


In [12]:
holiday_data.groupby("is_shopping_season").count()

,date,postal_code,is_holiday,holiday_name,days_since_last_holiday,days_until_next_holiday,state,is_major_holiday
is_shopping_season,,,,,,,,
0,1817,1817,1817,1817,1817,1817,1817,1817
1,13,13,13,13,13,13,13,13


In [22]:
from src.data.repositories import OrderRepository,CustomerRepository,AnalyticsRepository
from datetime import datetime, timedelta

In [23]:
order_repository = OrderRepository()
customer_repository = CustomerRepository()
analytics_repository = AnalyticsRepository()

In [32]:
order_items = order_repository.get_daily_sales(start_date=datetime.now() - timedelta(days=1), end_date=datetime.now())

In [34]:
order_items.head()

,date,location_id,order_count,customer_count,total_revenue,avg_order_value,new_customer_count,repeat_customer_count
0,2025-07-30,LVTJHAGAMD2ZY,67,55,999.53,11.622442,28,27
1,2025-07-30,LAFZJ3CHGB11Z,45,28,807.16,10.482597,11,17
2,2025-07-30,LWDYYDAMG4AH5,40,30,575.71,9.757797,14,16
3,2025-07-30,LMF86PJ371T9T,248,157,6948.06,14.942065,38,119
4,2025-07-30,L3T37TFC0S7WB,121,93,1650.83,9.597849,36,57


In [7]:
#!/usr/bin/env python3
"""
因果推断数据准备脚本
利用现有的connectors和repositories准备因果推断所需的数据
"""

import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
# from pathlib import Path

# 添加项目根目录到路径
# project_root = Path(__file__).parent.parent
# sys.path.append(str(project_root))

from src.data.repositories import OrderRepository, CustomerRepository
from src.data.connectors import ClickHouseConnector
from config.settings import Settings

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class CausalDataPreparator:
    """因果推断数据准备器"""

    def __init__(self):
        self.config = Settings()
        self.ch_connector = ClickHouseConnector()
        self.order_repo = OrderRepository()
        self.customer_repo = CustomerRepository()

    def prepare_causal_dataset(self, days_back: int = 180) -> pd.DataFrame:
        """
        准备因果推断数据集

        Args:
            days_back: 回溯天数，默认180天

        Returns:
            pandas.DataFrame: 准备好的数据集
        """
        logger.info(f"开始准备因果推断数据集，回溯{days_back}天")

        try:
            # 1. 获取基础数据
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days_back)

            # 获取销售数据
            logger.info("获取销售数据...")
            sales_data = self._get_sales_data(start_date, end_date)

            if sales_data.empty:
                logger.error("无法获取销售数据，流程终止")
                return pd.DataFrame()

            # 获取客户数据
            logger.info("获取客户数据...")
            customer_data = self._get_customer_data()

            # 获取商品数据
            logger.info("获取商品数据...")
            item_data = self._get_item_data()

            # 2. 数据合并和清理
            logger.info("合并和清理数据...")
            merged_data = self._merge_and_clean_data(sales_data, customer_data, item_data)

            if merged_data.empty:
                logger.error("数据合并后为空，流程终止")
                return pd.DataFrame()

            # 3. 创建处理变量
            logger.info("创建处理变量...")
            try:
                merged_data = self._create_treatment_variables(merged_data)
            except Exception as e:
                logger.error(f"创建处理变量失败: {e}")
                # 创建最基本的处理变量
                merged_data['has_promotion'] = 0
                merged_data['has_discount'] = 0
                merged_data['vip_treatment'] = 0

            # 4. 创建混淆变量
            logger.info("创建混淆变量...")
            try:
                merged_data = self._create_confounding_variables(merged_data)
            except Exception as e:
                logger.error(f"创建混淆变量失败: {e}")
                # 创建最基本的混淆变量
                merged_data['is_weekend'] = 0
                merged_data['hour_of_day'] = 12
                merged_data['location_encoded'] = 0
                merged_data['category_encoded'] = 0

            # 5. 创建结果变量
            logger.info("创建结果变量...")
            try:
                merged_data = self._create_outcome_variables(merged_data)
            except Exception as e:
                logger.error(f"创建结果变量失败: {e}")
                # 确保至少有基本的结果变量
                if 'revenue' in merged_data.columns:
                    merged_data['log_revenue'] = np.log1p(merged_data['revenue'])

            # 6. 最终数据验证
            logger.info("数据验证...")
            validated_data = self._validate_data(merged_data)

            if not validated_data.empty:
                logger.info(f"数据准备完成，共{len(validated_data)}条记录")
            else:
                logger.error("数据验证后为空")

            return validated_data

        except Exception as e:
            logger.error(f"数据准备过程中发生错误: {e}", exc_info=True)
            return pd.DataFrame()

    def _get_sales_data(self, start_date: datetime, end_date: datetime) -> pd.DataFrame:
        """获取销售数据"""
        try:
            # 使用现有的repository方法
            sales_data = self.order_repo.get_daily_sales(start_date, end_date)

            # 打印调试信息
            if not sales_data.empty:
                logger.info(f"从repository获取到数据，形状: {sales_data.shape}")
                logger.info(f"列名: {list(sales_data.columns)}")
                sales_data = self._standardize_column_names(sales_data)
            else:
                logger.info("Repository返回空数据，尝试直接查询...")
                # 尝试多种可能的查询
                sales_data = self._try_direct_queries(start_date, end_date)

            return sales_data

        except Exception as e:
            logger.error(f"获取销售数据失败: {e}")
            logger.info("尝试使用模拟数据...")
            return self._create_sample_data(start_date, end_date)

    def _try_direct_queries(self, start_date: datetime, end_date: datetime) -> pd.DataFrame:
        """尝试不同的直接查询方式"""

        # 查询1：完整字段查询
        queries = [
            """
            SELECT
                customer_id,
                item_name,
                category_name,
                location_id,
                order_final_total_amt as revenue,
                quantity,
                toDate(created_at_pt) as date,
                toHour(created_at_pt) as hour
            FROM dw.fact_order_item_variations
            WHERE created_at_pt >= '%(start_date)s'
                AND created_at_pt < '%(end_date)s'
                AND pay_status = 'COMPLETED'
            LIMIT 1000
            """,

            # 查询2：简化字段查询
            """
            SELECT
                *
            FROM dw.fact_order_item_variations
            WHERE created_at_pt >= '%(start_date)s'
                AND created_at_pt < '%(end_date)s'
            LIMIT 10
            """,

            # 查询3：表结构查询
            """
            DESCRIBE TABLE dw.fact_order_item_variations
            """
        ]

        for i, query in enumerate(queries):
            try:
                logger.info(f"尝试查询{i+1}...")
                formatted_query = query % {
                    'start_date': start_date.strftime('%Y-%m-%d'),
                    'end_date': end_date.strftime('%Y-%m-%d')
                }

                result = self.ch_connector.query_df(formatted_query)

                if not result.empty:
                    logger.info(f"查询{i+1}成功，形状: {result.shape}")
                    logger.info(f"列名: {list(result.columns)}")

                    if i == 2:  # 表结构查询
                        logger.info("表结构:")
                        for _, row in result.iterrows():
                            logger.info(f"  {row}")
                        continue
                    else:
                        return self._standardize_column_names(result)

            except Exception as e:
                logger.warning(f"查询{i+1}失败: {e}")
                continue

        return pd.DataFrame()

    def _standardize_column_names(self, data: pd.DataFrame) -> pd.DataFrame:
        """标准化列名"""
        # 列名映射字典
        column_mapping = {
            # customer相关
            'customerId': 'customer_id',
            'customer': 'customer_id',
            'user_id': 'customer_id',
            'userId': 'customer_id',

            # item相关
            'itemName': 'item_name',
            'product_name': 'item_name',
            'productName': 'item_name',

            # category相关
            'categoryName': 'category_name',
            'category': 'category_name',

            # location相关
            'locationId': 'location_id',
            'location': 'location_id',
            'store_id': 'location_id',

            # revenue相关
            'total_amount': 'revenue',
            'amount': 'revenue',
            'order_amount': 'revenue',
            'order_final_total_amt': 'revenue',

            # date相关
            'order_date': 'date',
            'created_date': 'date',
            'purchase_date': 'date'
        }

        # 应用映射
        data = data.rename(columns=column_mapping)

        # 如果还是没有customer_id，尝试创建
        if 'customer_id' not in data.columns:
            # 寻找可能的ID列
            id_columns = [col for col in data.columns if 'id' in col.lower()]
            if id_columns:
                logger.info(f"未找到customer_id，使用 {id_columns[0]} 作为customer_id")
                data['customer_id'] = data[id_columns[0]]
            else:
                logger.warning("未找到任何客户ID列，创建虚拟customer_id")
                # 创建虚拟customer_id
                data['customer_id'] = 'CUST_' + pd.Series(range(len(data))).astype(str)

        return data

    def _create_sample_data(self, start_date: datetime, end_date: datetime) -> pd.DataFrame:
        """创建示例数据用于测试"""
        logger.info("创建模拟数据进行测试...")

        np.random.seed(42)

        # 生成日期序列
        date_range = pd.date_range(start=start_date, end=end_date, freq='D')

        # 为每天生成多条记录
        data_list = []
        customer_ids = [f'CUST_{str(i).zfill(4)}' for i in range(1, 101)]  # 100个客户
        items = ['商品A', '商品B', '商品C', '商品D', '商品E']
        categories = ['电子产品', '服装', '食品', '家居', '图书']
        locations = ['店铺1', '店铺2', '店铺3']

        for date in date_range:
            # 每天随机生成10-50条记录
            daily_records = np.random.randint(10, 51)

            for _ in range(daily_records):
                data_list.append({
                    'customer_id': np.random.choice(customer_ids),
                    'item_name': np.random.choice(items),
                    'category_name': np.random.choice(categories),
                    'location_id': np.random.choice(locations),
                    'revenue': np.random.lognormal(4, 0.5),  # 对数正态分布生成价格
                    'quantity': np.random.randint(1, 5),
                    'date': date.date(),
                    'hour': np.random.randint(9, 22)  # 营业时间9-22点
                })

        sample_data = pd.DataFrame(data_list)
        logger.info(f"创建了 {len(sample_data)} 条模拟数据")

        return sample_data

    def _get_customer_data(self) -> pd.DataFrame:
        """获取客户数据"""
        try:
            # 使用现有的repository方法
            customer_data = self.customer_repo.get_customer_segments()

            if not customer_data.empty:
                logger.info(f"从repository获取客户数据，形状: {customer_data.shape}")
                logger.info(f"客户数据列名: {list(customer_data.columns)}")
                customer_data = self._standardize_column_names(customer_data)
            else:
                logger.info("Repository返回空客户数据，尝试直接查询...")
                # 尝试直接查询客户信息
                queries = [
                    """
                    SELECT
                        customer_id,
                        CASE
                            WHEN high_value_customer = 1 THEN 'high_value'
                            WHEN key_development_customer = 1 THEN 'key_development'
                            WHEN regular_customer = 1 THEN 'regular'
                            WHEN critical_win_back_customer = 1 THEN 'win_back'
                            ELSE 'other'
                        END as customer_segment,
                        order_final_total_amt as lifetime_value,
                        order_final_avg_amt as avg_order_value,
                        order_final_total_cnt as total_orders,
                        CASE WHEN high_value_customer = 1 THEN 1 ELSE 0 END as is_high_value
                    FROM ads.customer_profile
                    WHERE order_final_total_cnt > 0
                    LIMIT 1000
                    """,

                    """
                    SELECT * FROM ads.customer_profile LIMIT 10
                    """,

                    """
                    DESCRIBE TABLE ads.customer_profile
                    """
                ]

                for i, query in enumerate(queries):
                    try:
                        logger.info(f"尝试客户数据查询{i+1}...")
                        result = self.ch_connector.query_df(query)

                        if not result.empty:
                            logger.info(f"客户数据查询{i+1}成功，形状: {result.shape}")
                            logger.info(f"列名: {list(result.columns)}")

                            if i == 2:  # 表结构查询
                                logger.info("客户表结构:")
                                for _, row in result.iterrows():
                                    logger.info(f"  {row}")
                                continue
                            else:
                                customer_data = self._standardize_column_names(result)
                                break

                    except Exception as e:
                        logger.warning(f"客户数据查询{i+1}失败: {e}")
                        continue

            # 如果还是没有数据，返回空DataFrame
            if customer_data.empty:
                logger.warning("无法获取客户数据，将在后续步骤中创建默认客户信息")

            return customer_data

        except Exception as e:
            logger.error(f"获取客户数据失败: {e}")
            return pd.DataFrame()

    def _get_item_data(self) -> pd.DataFrame:
        """获取商品数据"""
        try:
            # 获取商品表现数据
            item_data = self.order_repo.get_item_performance(days=30)

            if not item_data.empty:
                logger.info(f"从repository获取商品数据，形状: {item_data.shape}")
                logger.info(f"商品数据列名: {list(item_data.columns)}")
                item_data = self._standardize_column_names(item_data)
            else:
                logger.info("Repository返回空商品数据，尝试直接查询...")
                # 直接查询商品信息
                queries = [
                    """
                    SELECT
                        item_name,
                        category_name,
                        AVG(order_final_total_amt) as avg_item_revenue,
                        COUNT(DISTINCT customer_id) as unique_buyers,
                        SUM(quantity) as total_quantity
                    FROM dw.fact_order_item_variations
                    WHERE created_at_pt >= today() - 30
                        AND pay_status = 'COMPLETED'
                    GROUP BY item_name, category_name
                    LIMIT 100
                    """,

                    """
                    SELECT DISTINCT item_name, category_name
                    FROM dw.fact_order_item_variations
                    LIMIT 50
                    """
                ]

                for i, query in enumerate(queries):
                    try:
                        logger.info(f"尝试商品数据查询{i+1}...")
                        result = self.ch_connector.query_df(query)

                        if not result.empty:
                            logger.info(f"商品数据查询{i+1}成功，形状: {result.shape}")
                            item_data = self._standardize_column_names(result)
                            break

                    except Exception as e:
                        logger.warning(f"商品数据查询{i+1}失败: {e}")
                        continue

            return item_data

        except Exception as e:
            logger.error(f"获取商品数据失败: {e}")
            return pd.DataFrame()

    def _merge_and_clean_data(self, sales_data: pd.DataFrame,
                             customer_data: pd.DataFrame,
                             item_data: pd.DataFrame) -> pd.DataFrame:
        """合并和清理数据"""
        if sales_data.empty:
            logger.warning("销售数据为空，无法继续")
            return pd.DataFrame()

        logger.info(f"开始合并数据 - 销售数据: {sales_data.shape}")
        merged_data = sales_data.copy()

        # 确保基本字段存在
        if 'customer_id' not in merged_data.columns:
            logger.error("销售数据中缺少customer_id字段")
            return pd.DataFrame()

        if 'revenue' not in merged_data.columns:
            # 尝试寻找收入相关字段
            revenue_cols = [col for col in merged_data.columns if any(keyword in col.lower() for keyword in ['amount', 'revenue', 'total', 'price'])]
            if revenue_cols:
                merged_data['revenue'] = merged_data[revenue_cols[0]]
                logger.info(f"使用 {revenue_cols[0]} 作为revenue字段")
            else:
                logger.error("无法找到收入相关字段")
                return pd.DataFrame()

        # 合并客户数据
        if not customer_data.empty and 'customer_id' in customer_data.columns:
            logger.info(f"合并客户数据: {customer_data.shape}")
            try:
                merged_data = merged_data.merge(
                    customer_data,
                    on='customer_id',
                    how='left'
                )
                logger.info(f"合并后形状: {merged_data.shape}")
            except Exception as e:
                logger.warning(f"客户数据合并失败: {e}")
        else:
            logger.info("客户数据为空或缺少customer_id，创建默认客户分群")
            # 创建默认客户分群
            merged_data['customer_segment'] = 'regular'
            merged_data['is_high_value'] = 0

        # 合并商品数据
        if not item_data.empty:
            merge_keys = []
            if 'item_name' in merged_data.columns and 'item_name' in item_data.columns:
                merge_keys.append('item_name')
            if 'category_name' in merged_data.columns and 'category_name' in item_data.columns:
                merge_keys.append('category_name')

            if merge_keys:
                logger.info(f"使用 {merge_keys} 合并商品数据")
                try:
                    merged_data = merged_data.merge(
                        item_data[merge_keys + ['avg_item_revenue']] if 'avg_item_revenue' in item_data.columns else item_data[merge_keys],
                        on=merge_keys,
                        how='left'
                    )
                    logger.info(f"商品数据合并后形状: {merged_data.shape}")
                except Exception as e:
                    logger.warning(f"商品数据合并失败: {e}")

        # 数据清理
        logger.info("开始数据清理...")
        original_count = len(merged_data)

        # 移除缺失关键字段的记录
        required_fields = ['customer_id', 'revenue']
        available_fields = [field for field in required_fields if field in merged_data.columns]

        if available_fields:
            merged_data = merged_data.dropna(subset=available_fields)
            logger.info(f"移除缺失值后: {len(merged_data)} 条记录 (原始: {original_count})")

        # 移除异常值（比如负收入或0收入）
        if 'revenue' in merged_data.columns:
            before_filter = len(merged_data)
            merged_data = merged_data[merged_data['revenue'] > 0]
            logger.info(f"移除异常收入后: {len(merged_data)} 条记录 (过滤前: {before_filter})")

        # 填充缺失值
        if 'location_id' in merged_data.columns:
            merged_data['location_id'] = merged_data['location_id'].fillna('unknown')
        else:
            merged_data['location_id'] = 'unknown'

        if 'category_name' in merged_data.columns:
            merged_data['category_name'] = merged_data['category_name'].fillna('other')
        else:
            merged_data['category_name'] = 'other'

        if 'item_name' in merged_data.columns:
            merged_data['item_name'] = merged_data['item_name'].fillna('unknown_item')
        else:
            merged_data['item_name'] = 'unknown_item'

        logger.info(f"数据清理完成，最终形状: {merged_data.shape}")
        return merged_data

    def _create_treatment_variables(self, data: pd.DataFrame) -> pd.DataFrame:
        """创建处理变量"""
        # 1. 促销活动标记（基于收入异常检测）
        # 计算每日平均收入
        daily_revenue = data.groupby('date')['revenue'].sum().reset_index()
        daily_avg = daily_revenue['revenue'].mean()
        daily_std = daily_revenue['revenue'].std()

        # 高收入日期可能有促销活动
        promotion_threshold = daily_avg + 1.5 * daily_std
        promotion_dates = daily_revenue[daily_revenue['revenue'] > promotion_threshold]['date'].tolist()

        data['has_promotion'] = data['date'].isin(promotion_dates).astype(int)

        # 2. 折扣活动标记（基于低于平均商品价格）
        if 'avg_item_revenue' in data.columns:
            data['has_discount'] = (data['revenue'] < data['avg_item_revenue'] * 0.8).astype(int)
        else:
            data['has_discount'] = 0

        # 3. 会员专享活动（针对高价值客户）
        data['vip_treatment'] = ((data.get('is_high_value', 0) == 1) &
                                (data['has_promotion'] == 1)).astype(int)

        return data

    def _create_confounding_variables(self, data: pd.DataFrame) -> pd.DataFrame:
        """创建混淆变量"""
        # 1. 时间相关变量
        data['date'] = pd.to_datetime(data['date'])
        data['is_weekend'] = data['date'].dt.dayofweek.isin([5, 6]).astype(int)
        data['month'] = data['date'].dt.month
        data['day_of_week'] = data['date'].dt.dayofweek

        if 'hour' in data.columns:
            data['hour_of_day'] = data['hour']
        else:
            data['hour_of_day'] = 12  # 默认中午

        # 2. 地理位置
        data['location_encoded'] = pd.Categorical(data['location_id']).codes

        # 3. 商品类别
        data['category_encoded'] = pd.Categorical(data['category_name']).codes

        # 4. 客户特征
        data['customer_segment_encoded'] = pd.Categorical(
            data.get('customer_segment', 'regular')
        ).codes

        # 5. 历史购买行为（如果有的话）
        if 'total_orders' in data.columns:
            data['is_frequent_buyer'] = (data['total_orders'] > data['total_orders'].median()).astype(int)
        else:
            data['is_frequent_buyer'] = 0

        return data

    def _create_outcome_variables(self, data: pd.DataFrame) -> pd.DataFrame:
        """创建结果变量"""
        # 1. 主要结果变量 - 收入
        data['log_revenue'] = np.log1p(data['revenue'])  # 对数变换减少偏态

        # 2. 订单量（如果有quantity字段）
        if 'quantity' in data.columns:
            data['order_quantity'] = data['quantity']
        else:
            data['order_quantity'] = 1  # 默认每个记录为1个订单

        # 3. 客户价值相关
        if 'avg_order_value' in data.columns:
            data['customer_value_change'] = data['revenue'] - data['avg_order_value']
        else:
            customer_avg = data.groupby('customer_id')['revenue'].transform('mean')
            data['customer_value_change'] = data['revenue'] - customer_avg

        return data

    def _validate_data(self, data: pd.DataFrame) -> pd.DataFrame:
        """验证数据质量"""
        if data.empty:
            logger.error("数据集为空，无法进行验证")
            return data

        logger.info(f"数据集基本信息:")
        logger.info(f"- 总记录数: {len(data)}")
        logger.info(f"- 总列数: {len(data.columns)}")
        logger.info(f"- 列名: {list(data.columns)}")

        # 检查customer_id
        if 'customer_id' in data.columns:
            logger.info(f"- 唯一客户数: {data['customer_id'].nunique()}")
        else:
            logger.warning("- 缺少customer_id字段")

        # 检查日期范围
        if 'date' in data.columns:
            try:
                data['date'] = pd.to_datetime(data['date'])
                logger.info(f"- 日期范围: {data['date'].min()} 到 {data['date'].max()}")
            except Exception as e:
                logger.warning(f"日期字段处理失败: {e}")
        else:
            logger.warning("- 缺少date字段")

        # 检查关键变量
        required_vars = ['customer_id', 'revenue']
        available_vars = [var for var in required_vars if var in data.columns]
        missing_vars = [var for var in required_vars if var not in data.columns]

        if available_vars:
            logger.info(f"- 可用关键变量: {available_vars}")
        if missing_vars:
            logger.warning(f"- 缺少关键变量: {missing_vars}")

        # 检查处理变量分布
        if 'has_promotion' in data.columns:
            promo_dist = data['has_promotion'].value_counts()
            logger.info(f"- 促销分布: {promo_dist.to_dict()}")
        else:
            logger.info("- 促销变量尚未创建")

        # 检查收入分布
        if 'revenue' in data.columns:
            logger.info(f"- 收入统计: 均值={data['revenue'].mean():.2f}, 中位数={data['revenue'].median():.2f}")
            logger.info(f"- 收入范围: {data['revenue'].min():.2f} - {data['revenue'].max():.2f}")

        # 移除完全重复的记录
        before_count = len(data)
        data = data.drop_duplicates()
        after_count = len(data)

        if before_count != after_count:
            logger.info(f"- 移除了 {before_count - after_count} 条重复记录")

        # 检查是否有足够的数据进行因果推断
        if len(data) < 100:
            logger.warning("数据量较少，可能影响因果推断的准确性")
        elif len(data) < 1000:
            logger.info("数据量适中，可以进行基础因果推断")
        else:
            logger.info("数据量充足，可以进行稳健的因果推断")

        return data

    def save_dataset(self, data: pd.DataFrame, output_path: str = "data/causal_dataset.csv") -> None:
        """保存数据集"""
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        data.to_csv(output_path, index=False)
        logger.info(f"数据集已保存到: {output_path}")

        # 保存数据字典
        dict_path = output_path.replace('.csv', '_dictionary.txt')
        with open(dict_path, 'w', encoding='utf-8') as f:
            f.write("因果推断数据集字典\n")
            f.write("=" * 50 + "\n\n")

            f.write("处理变量 (Treatment Variables):\n")
            f.write("- has_promotion: 是否有促销活动 (0/1)\n")
            f.write("- has_discount: 是否有折扣 (0/1)\n")
            f.write("- vip_treatment: VIP专享活动 (0/1)\n\n")

            f.write("结果变量 (Outcome Variables):\n")
            f.write("- revenue: 收入金额\n")
            f.write("- log_revenue: 对数收入\n")
            f.write("- order_quantity: 订单数量\n\n")

            f.write("混淆变量 (Confounding Variables):\n")
            f.write("- is_weekend: 是否周末 (0/1)\n")
            f.write("- hour_of_day: 小时\n")
            f.write("- location_encoded: 地理位置编码\n")
            f.write("- category_encoded: 商品类别编码\n")
            f.write("- customer_segment_encoded: 客户分群编码\n")

        logger.info(f"数据字典已保存到: {dict_path}")


def main():
    """主函数"""
    try:
        logger.info("开始初始化数据准备器...")

        # 初始化数据准备器
        try:
            preparator = CausalDataPreparator()
            logger.info("数据准备器初始化成功")
        except Exception as e:
            logger.error(f"数据准备器初始化失败: {e}")
            logger.info("这可能是因为配置文件或数据库连接问题")
            return

        # 准备数据集
        logger.info("开始准备数据集...")
        dataset = preparator.prepare_causal_dataset(days_back=180)

        if not dataset.empty:
            # 保存数据集
            logger.info("保存数据集...")
            preparator.save_dataset(dataset)

            # 显示基本统计
            print("\n" + "="*50)
            print("数据集准备完成!")
            print("="*50)
            print(f"数据形状: {dataset.shape}")
            print(f"列名: {list(dataset.columns)}")

            # 显示数据预览
            print(f"\n前5行预览:")
            print(dataset.head())

            # 显示数据类型
            print(f"\n数据类型:")
            print(dataset.dtypes)

            # 显示缺失值统计
            print(f"\n缺失值统计:")
            missing_stats = dataset.isnull().sum()
            print(missing_stats[missing_stats > 0])

            print(f"\n数据已保存到: data/causal_dataset.csv")
            print(f"数据字典已保存到: data/causal_dataset_dictionary.txt")

        else:
            logger.error("数据集为空，请检查以下可能的问题:")
            logger.error("1. 数据库连接是否正常")
            logger.error("2. 表名和字段名是否正确")
            logger.error("3. 数据是否存在于指定的时间范围内")
            logger.error("4. 权限是否足够访问相关表")

    except Exception as e:
        logger.error(f"数据准备失败: {e}", exc_info=True)
        logger.info("\n调试建议:")
        logger.info("1. 检查config/config.json中的数据库配置")
        logger.info("2. 确认数据库表是否存在并可访问")
        logger.info("3. 检查网络连接和防火墙设置")
        logger.info("4. 查看完整的错误堆栈信息")


if __name__ == "__main__":
    main()

2025-07-31 14:05:14,334 - __main__ - INFO - 开始初始化数据准备器...
2025-07-31 14:05:14,335 - config.settings - INFO - Config loaded from /Users/xander/PycharmProjects/FBR_AI_Agine/config/config.json
2025-07-31 14:05:14,337 - __main__ - INFO - 数据准备器初始化成功
2025-07-31 14:05:14,337 - __main__ - INFO - 开始准备数据集...
2025-07-31 14:05:14,337 - __main__ - INFO - 开始准备因果推断数据集，回溯180天
2025-07-31 14:05:14,337 - __main__ - INFO - 获取销售数据...
2025-07-31 14:05:16,662 - src.data.connectors - INFO - Connected to ClickHouse using clickhouse_connect: clickhouse-0-0.umetea.net:443
2025-07-31 14:05:17,577 - __main__ - INFO - 从repository获取到数据，形状: (4061, 8)
2025-07-31 14:05:17,578 - __main__ - INFO - 列名: ['date', 'location_id', 'order_count', 'customer_count', 'total_revenue', 'avg_order_value', 'new_customer_count', 'repeat_customer_count']
2025-07-31 14:05:17,583 - __main__ - INFO - 未找到customer_id，使用 location_id 作为customer_id
2025-07-31 14:05:17,585 - __main__ - INFO - 获取客户数据...
2025-07-31 14:05:21,732 - src.data.connecto